In [1]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the pre-trained models
cnn_model = load_model('/kaggle/input/models/CNN.h5')
lstm_model = load_model('/kaggle/input/models/LSTM.h5')

# Define the input shape and number of classes
INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = 8

# Define the batch size
batch_size = 32

# Create data generators for the training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/preprocessed-dataset/dataset-split/train/',
        target_size=INPUT_SHAPE[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        '/kaggle/input/preprocessed-dataset/dataset-split/val/',
        target_size=INPUT_SHAPE[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '/kaggle/input/preprocessed-dataset/dataset-split/test/',
        target_size=INPUT_SHAPE[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

# Fit the CNN model
cnn_model.fit(train_generator, epochs=50, validation_data=val_generator, verbose=0)

# Fit the LSTM model
lstm_model.fit(train_generator, epochs=50, validation_data=val_generator, verbose=0)

# Make predictions on the validation and test sets using the CNN and LSTM models
val_cnn_preds = cnn_model.predict(val_generator)
test_cnn_preds = cnn_model.predict(test_generator)

val_lstm_preds = lstm_model.predict(val_generator)
test_lstm_preds = lstm_model.predict(test_generator)

# Concatenate the CNN and LSTM predictions for the validation and test sets
val_preds = np.concatenate((val_cnn_preds, val_lstm_preds), axis=1)
test_preds = np.concatenate((test_cnn_preds, test_lstm_preds), axis=1)

# Train an AdaBoost classifier on the concatenated predictions
base_estimator = DecisionTreeClassifier(max_depth=1)
ada_clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100)
ada_clf.fit(val_preds, val_generator.labels)

# Use the ensemble to make predictions on the test set
ensemble_preds = ada_clf.predict(test_preds)
ensemble_acc = accuracy_score(test_generator.labels, ensemble_preds)

print('Ensemble test accuracy:', ensemble_acc)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found 3960 images belonging to 8 classes.
Found 1130 images belonging to 8 classes.
Found 575 images belonging to 8 classes.
18/18 [==============================] - 4s 228ms/step
Ensemble test accuracy: 0.1408695652173913
